In [99]:
import requests
import json
import os
import foursquare
import pandas as pd
from tqdm import tqdm

# Foursquare

### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

#### Step 1: Set code to review full API

In [96]:
# Setup Foursquare API credentials 
api_key = os.environ.get('FOURSQUARE_API_KEY')

# Define the endpoint URL for the Foursqare request
endpoint = "https://api.foursquare.com/v3/places/search"

# Define the query parameters to include only the latitude and longitude fields
def foursquare_query(latitude, longitude):
    params = {
        'll': f'{latitude},{longitude}', 'radius':'1000'
        }
    #Headers specific to FourSquare - used to specify wanted metadata - not typically found under all APIs. Found under https://location.foursquare.com/developer/reference/authentication
    headers = {
        "Accept": "application/json",
        "Authorization": api_key
        }

    # Send an HTTP GET request to the endpoint URL with the query parameters
    response = requests.get(endpoint, params=params, headers=headers)

    # Get the JSON content from the response
    json_content = response.json()

    # Print out the entire JSON response to see catergory names
    return json_content



In [ ]:
#### Step 2: Retrieve Foursquare results

In [113]:
#Retrieve the Foursquare query results for a given latitude and longitude
def bike_radius(latitude, longitude):
    json_content = foursquare_query(latitude, longitude)
    poi = json_content['results']
    return json_content

#extract the relevant information from the Foursquare query result and store it in a dictionary that is returned as a list.
def bike_radius_info(latitude, longitude, location_name):
    json_content = foursquare_query(latitude, longitude)
    poi_list = []
    for poi in json_content['results']:
        poi_dict = {}
        poi_dict['fsq_id'] = poi['fsq_id']
        poi_dict['location_name'] = location_name
        poi_dict['poi_type'] = [category['name'] for category in poi['categories']]
        poi_list.append(poi_dict)
    return poi_list

In [114]:
# bike_radius('51.529163', '-0.109971')
# read the file containing the locations
bike_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes1_Data.csv')

# extract the name, latitude, and longitude columns from the City Bike Data
names = bike_df['name']
latitudes = bike_df['latitude']
longitudes = bike_df['longitude']


# loop through each location and call the bike_radius_info function
for i in tqdm(range(len(latitudes))):
    fsq_ids = bike_radius_info(latitudes[i], longitudes[i], names[i])
    for fsq_id in fsq_ids:
        print(fsq_id['fsq_id'], fsq_id['location_name'], fsq_id['poi_type'])


  0%|                                           | 1/795 [00:01<15:59,  1.21s/it]

4bd6e9305631c9b6fa8ca630 001023 - River Street , Clerkenwell ['Park']
4ac518cdf964a520cca520e3 001023 - River Street , Clerkenwell ['Rock Club', 'Lounge', 'Restaurant']
4cb5ee61f6616dcbfe42db08 001023 - River Street , Clerkenwell ['Pub']
5555d73f498eba1d92b0bd0a 001023 - River Street , Clerkenwell ['Bakery', 'Coffee Shop', 'Restaurant']
5777b7f6498ec5cc16dce198 001023 - River Street , Clerkenwell ['Mediterranean Restaurant', 'Middle Eastern Restaurant']
4ac518d5f964a520f0a720e3 001023 - River Street , Clerkenwell ['Mediterranean Restaurant', 'Middle Eastern Restaurant', 'Tapas Restaurant']
4bffb0f0daf9c9b6c886faef 001023 - River Street , Clerkenwell ['Food Truck']
4c1a254898f4a593c71701f6 001023 - River Street , Clerkenwell ['Landmarks and Outdoors']
4c7fc0941415b713dd39035d 001023 - River Street , Clerkenwell ['Tapas Restaurant']
51d48bf0498ed6595ad3c1be 001023 - River Street , Clerkenwell ['Restaurant']


  0%|                                           | 2/795 [00:01<12:13,  1.08it/s]

4b1a6a68f964a5200dea23e3 001018 - Phillimore Gardens, Kensington ['Café', 'Coffee Shop', 'Diner']
4f312695e4b0951ff459b9d5 001018 - Phillimore Gardens, Kensington ['Restaurant']
4ac518d1f964a520eca620e3 001018 - Phillimore Gardens, Kensington ['Museum']
4ca99c3bb0b8236a1bb7bee6 001018 - Phillimore Gardens, Kensington ['Café', 'Coffee Shop', 'French Restaurant']
4bb899ef98c7ef3b33343102 001018 - Phillimore Gardens, Kensington ['Travel Agency']
4b83d8b7f964a520571331e3 001018 - Phillimore Gardens, Kensington ['Thai Restaurant']
57d5606a498e432cab0a7099 001018 - Phillimore Gardens, Kensington ['History Museum']
4b6d8a7af964a520537b2ce3 001018 - Phillimore Gardens, Kensington ['English Restaurant', 'Brasserie', 'Modern European Restaurant']
56759402498eed3ca733d6a1 001018 - Phillimore Gardens, Kensington ['Retail']
4b181c45f964a52014cd23e3 001018 - Phillimore Gardens, Kensington ['Italian Restaurant']


  0%|▏                                          | 3/795 [00:02<12:08,  1.09it/s]

...

4b38e190f964a520a45125e3 001074 - Newgate Street , St. Paul's ['Bakery', 'French Restaurant']
5c913099b04056002caa8e53 001074 - Newgate Street , St. Paul's ['Pizzeria']
4af9c2e5f964a520001422e3 001074 - Newgate Street , St. Paul's ['Burger Joint']
4d9f608b0ea841bde47adbd7 001074 - Newgate Street , St. Paul's ['Grocery Store / Supermarket']
4b62d706f964a520e2542ae3 001074 - Newgate Street , St. Paul's ['Coffee Shop']
51372f42e4b00a9496994d3b 001074 - Newgate Street , St. Paul's ['Falafel Restaurant', 'Fast Food Restaurant', 'Middle Eastern Restaurant']
4c404717d691c9b67be38a0a 001074 - Newgate Street , St. Paul's ['Fast Food Restaurant', 'Italian Restaurant']
562fc107498e30b20a0244cf 001074 - Newgate Street , St. Paul's ['Fast Food Restaurant', 'Mediterranean Restaurant']
4c52fd16d797e21ece366b7e 001074 - Newgate Street , St. Paul's ['Barbershop']
4d527eb7d7eaa1437dd97e0f 001074 - Newgate Street , St. Paul's ['Bank']


100%|████████████████████████████████████████▉| 794/795 [10:39<00:00,  1.36it/s]

5c695c4ffebf31002c2054d8 003469 - Cadogan Place, Knightsbridge ['Ice Cream Parlor']
59b5297af62e094c148fe918 003469 - Cadogan Place, Knightsbridge ['Coffee Shop', 'Restaurant']
59ea37c8356b491e436b04a4 003469 - Cadogan Place, Knightsbridge ['Vegan and Vegetarian Restaurant']
4ac518f0f964a520bcad20e3 003469 - Cadogan Place, Knightsbridge ['Clothing Store', "Men's Store"]
4c0a6f3232daef3b06a44b50 003469 - Cadogan Place, Knightsbridge ['Property Management Office', 'Real Estate Agency']
502e2b66e4b02982258e301f 003469 - Cadogan Place, Knightsbridge ['Retail']
4ac518e7f964a5205aab20e3 003469 - Cadogan Place, Knightsbridge ['Concert Hall', 'Music Venue', 'Restaurant']
5cf0e283270ee7002c0ea7b6 003469 - Cadogan Place, Knightsbridge ['Cafes, Coffee, and Tea Houses', 'Sandwich Restaurant']
527a8644498ec716be9d5129 003469 - Cadogan Place, Knightsbridge ['Cocktail Bar']
4ac518eef964a520f5ac20e3 003469 - Cadogan Place, Knightsbridge ['Baby Store']


100%|█████████████████████████████████████████| 795/795 [10:40<00:00,  1.24it/s]

4ac518dcf964a5204aa920e3 003493 - South Audley Street, Mayfair ['Chinese Restaurant']
4be2c8711dd22d7fb8c694bd 003493 - South Audley Street, Mayfair ['Night Club', 'Restaurant']
4bed2b6a91380f47617c9f18 003493 - South Audley Street, Mayfair ['Tobacco Store']
5980dbf30ff4f9479f731d60 003493 - South Audley Street, Mayfair ['French Restaurant']
5ce7fcd100b068002d2aa51f 003493 - South Audley Street, Mayfair ['Pastry Shop', 'Restaurant']
4e9b11e94690c21335507a89 003493 - South Audley Street, Mayfair ['Spa']
4e8d4f95e5e89c46f5bf0299 003493 - South Audley Street, Mayfair ['Steakhouse']
4ac518e3f964a520c6aa20e3 003493 - South Audley Street, Mayfair ['Lounge', 'Lebanese Restaurant', 'Moroccan Restaurant']
50896c8be4b0f94297081043 003493 - South Audley Street, Mayfair ['Coffee Shop']
4b7a924ff964a520ad312fe3 003493 - South Audley Street, Mayfair ['Coffee Shop']


In [105]:



# restaurant_info=[]
# landmarks_info
# # Write the for loop with my bike radius info (each time grabbing correct longitude and latitude)

# result= bike_radius_info(longitude, latitude)
# restaurant_info.append(result)
# landmarks_info.append(result)


# #Convert to DF
# # foursquare_info = pd.DataFrame(foursquare_info)
# # foursquare_info.head()


# restaurants=[]
# landmarks=[]
# # Write the for loop with my bike radius info (each time grabbing correct longitude and latitude)

# result= bike_radius_info
# restaurants.append(result)
# landmarks.append(result)

# # restaurants
# # # #Convert to DF
# # # restaurants = pd.DataFrame(foursquare_info)
# # # foursquare_info.head()

In [77]:

def bike_radius_info(latitude, longitude):
    json_content = foursquare_query(latitude, longitude)
    locations = []
    for result in json_content['results']:
        location = {}
        location['name'] = result['name']
        location['latitude'] = result['geocodes']['main']['latitude']
        location['longitude'] = result['geocodes']['main']['longitude']
        location['address'] = result['location']['formatted_address']
        location['categories'] = [cat['name'] for cat in result['categories']]
        locations.append(location)
    return locations

### Put your parsed results into a DataFrame

# Yelp

### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

##### Set code to review full API

In [ ]:
https://api.yelp.com/v3

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating